In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import skimage.io as io
import skimage.transform as trans
import shutil

from skimage import data

In [6]:
def create_dir_tree(directory, names):
    '''
    Crea directorio con n muestras,
    '''
    if os.path.exists(directory):
        shutil.rmtree(directory) # borramos directorio y lurgo le creamos
        
    os.mkdir(directory)
        
    for name in names:
        sample = os.path.join(directory,name)
        os.mkdir(sample)

In [43]:
random_names = pd.read_csv('random_names.csv')
random_names = random_names['Names'][:len(os.listdir("CASIA-IrisV1"))]
new_dir = "CASIA-Iris"
create_dir_tree(new_dir,random_names)

In [42]:
shutil.rmtree(new_dir)

In [44]:
def copy_samples(to_dir, from_dir):
    directories = os.listdir(to_dir) # nombres de las muestras
    i = 0
    k = 1
    for root, dirs, files in os.walk(from_dir):
        for file in files:
            if file.endswith(".bmp"):
                source = os.path.join(root, file)
                dest = os.path.join(to_dir,directories[i])
                shutil.copy(source, dest)
                if len(os.listdir(dest)) == 7:
                    i = i + 1
                    continue

In [77]:
copy_samples(new_dir, "CASIA-IrisV1")

In [78]:
# cambiamos nombre a las muestras
for root, dirs, files in os.walk(new_dir):
    for d in dirs:
        name = str(d)
        k = 1
        for i in os.listdir(os.path.join(new_dir,d)):
            os.rename(os.path.join(os.path.join(new_dir,d),i), os.path.join(os.path.join(new_dir,d),name + '_' + str(k)+'.bmp'))
            k += 1

In [4]:
def get_paths(directory):
    '''
    Devuelve la ruta relativa de las muestras .bmp
    '''
    paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".bmp"):
                paths.append(os.path.join(root, file))
    return paths

In [18]:
paths_casia = get_paths("CASIA-IrisV1")
original_samples = [cv2.imread(muestra,0) for muestra in paths_casia] # array de las muestras originales en b&w

In [14]:
def testGenerator(images, target_size= (320,320)):
    for muestra in images:
        img = muestra / 255
        img = trans.resize(img,target_size)
        img = np.reshape(img,(1,)+img.shape)
        yield img

In [15]:
def saveResult(results, save_dir, names):
    '''
    Guarda los imágenes segmentadas resultado de la red preentrenada
    
    Parámetros:
    results -- resultado de la red preentrenada
    save_path -- ruta en la que se guardarán las muestras segmentadas
    name_path -- directorio del que cogeremos los nombres de las muestras
    '''
    dim = (320, 280) # dimensiones originales de las muestras del dataset
    if os.path.exists(save_dir):
        shutil.rmtree(save_dir)
        
    os.mkdir(save_dir)
    for i,item in enumerate(results):
        img = item[:,:,0]
        img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
        io.imsave(os.path.join(save_dir,names[i]),img)

In [16]:
from keras.models import load_model

model = load_model('Iris_unet_d5.h5')
# model.summary()

In [19]:
gen = testGenerator(original_samples)
results = model.predict(gen,verbose=1)

756/756 [==============================] - 1054s 1s/step


In [46]:
samples_names = [file for file in os.listdir("CASIA-Iris")]
list(random_names)

['Serenity',
 'Cataleya',
 'Andrew',
 'Corey',
 'Maite',
 'Shira',
 'Michelle',
 'Zahra',
 'Keily',
 'Rivkah',
 'Gavriel',
 'Rafael',
 'Evangeline',
 'Yandel',
 'Mariam',
 'Chad',
 'Imani',
 'Shaina',
 'Erick',
 'Gabriella',
 'Naomi',
 'Atara',
 'Brendan',
 'Oumar',
 'Anas',
 'Carlos',
 'Aiza',
 'Malek',
 'Xin',
 'Brianny',
 'Cooper',
 'Avital',
 'Samiyah',
 'Franklin',
 'Andrea',
 'Tianna',
 'Youssef',
 'Zion',
 'Arlette',
 'Eliyahu',
 'Alice',
 'Ashton',
 'Ananya',
 'Nyah',
 'Emma',
 'Aharon',
 'Mariana',
 'Amayah',
 'Reed',
 'Jeancarlos',
 'Leibish',
 'Batsheva',
 'Damani',
 'Aleah',
 'Michal',
 'Mushka',
 'Pessy',
 'Aminata',
 'Janiyah',
 'Liz',
 'Zaiden',
 'Alvin',
 'Freya',
 'Hope',
 'Yahya',
 'Precious',
 'Nico',
 'Leora',
 'Avery',
 'Axel',
 'Helena',
 'Delilah',
 'Noam',
 'Jaelynn',
 'Margot',
 'Amberly',
 'Armaan',
 'Jolie',
 'Kacper',
 'Chany',
 'Lindsay',
 'Aidan',
 'Morgan',
 'Nehemiah',
 'Jaxon',
 'Devorah',
 'Frimet',
 'Arye',
 'Brucha',
 'Shayna',
 'Sasha',
 'Camren',
 

In [ ]:
# tiene que haber un directorio con las 756 muestras segmentadas
samples_names = [file for file in os.listdir("CASIA-Polar")]
saveResult(results, "U-net Output", samples_names)